In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


In [2]:
from typing import Callable, Optional, Union

import gymnasium as gym
import ray
from ray import tune
from ray.rllib.algorithms import ppo, AlgorithmConfig
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env

from sustaingym.envs.evcharging import EVChargingEnv, RealTraceGenerator, GMMsTraceGenerator, DiscreteActionWrapper
from sustaingym.envs.evcharging.event_generation import AbstractTraceGenerator
from sustaingym.envs.evcharging.utils import \
    DATE_FORMAT, DEFAULT_PERIOD_TO_RANGE, DATE_FORMAT, SiteStr

from gymnasium.wrappers import TimeLimit


###
NUM_SUBPROCESSES = 4
TIMESTEPS = 250_000
EVAL_FREQ = 10_000
SAMPLE_EVAL_PERIODS = {
    'Summer 2019':   ('2019-07-01', '2019-07-14'),
    'Fall 2019':     ('2019-11-04', '2019-11-17'),
    'Spring 2020':   ('2020-04-06', '2020-04-19'),
    'Summer 2021':   ('2021-07-05', '2021-07-18'),
}

def get_env(full: bool, real_trace: bool, dp: str, site: SiteStr, discrete: bool = False, seed: int=None) -> Callable:
    """Return environment.

    Args:
        full: if True, use full season; otherwise, use sample 2 weeks
        real_trace: choice of generator
        dp: 'Summer 2019', 'Fall 2019', 'Spring 2020', 'Summer 2021'
        site: 'caltech' or 'jpl'
        discrete: whether to wrap environment in discrete action wrapper
        seed: seed for GMMs generator
    
    Returns:
        Callable of environment
    """
    date_period = DEFAULT_PERIOD_TO_RANGE[dp] if full else SAMPLE_EVAL_PERIODS[dp]

    def _get_env() -> EVChargingEnv:
        if real_trace:
            gen: AbstractTraceGenerator = RealTraceGenerator(site, date_period)
        else:
            gen = GMMsTraceGenerator(site, date_period, seed=seed)
        
        if discrete:
            return TimeLimit(DiscreteActionWrapper(EVChargingEnv(gen)), max_episode_steps=288)
        else:
            return TimeLimit(EVChargingEnv(gen), max_episode_steps=288)
    return _get_env

In [3]:
from ray import tune
from ray.air import session
from ray.rllib.algorithms import ppo, AlgorithmConfig

# ray.init(num_cpus=3)
register_env("my_env", lambda config: get_env(**config)())

train_config = (
    ppo.PPOConfig()
    .environment("my_env", env_config={
        "full": True,
        "real_trace": False,
        "dp": "Summer 2019",
        "site": "caltech",
        "discrete": False,
        "seed": 123
    })
    .framework("tf2")
)
algo = train_config.build(env="my_env")

2023-04-06 04:43:52,816	INFO worker.py:1553 -- Started a local Ray instance.
(RolloutWorker pid=38135) 2023-04-06 04:43:59,003	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=38135) 2023-04-06 04:43:59,003	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2023-04-06 04:44:01,653	INFO trainable.py:172 -- Trainable.setup took 10.792 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [4]:
from sustaingym.algorithms.evcharging.baselines import RLLibAlgorithm

env = get_env(full=False, real_trace=True, dp='Summer 2019', site='caltech', discrete=False, seed=True)()
rllib_algo = RLLibAlgorithm(env, algo)
reward_breakdown = rllib_algo.run(14).to_dict('list')

env observation space:  (146,)


100%|██████████| 14/14 [01:46<00:00,  7.63s/it]


In [2]:


###

from ray import tune
from ray.air import session


def trainable(config: dict):
    checkpoint_dir = tune.get_trial_dir()
    print(checkpoint_dir)
    print(config)

def trainable(config: dict):
    checkpoint_dir = tune.get_trial_dir()
    print(checkpoint_dir)
    print(config)

    train_config = (
        ppo.PPOConfig()
        .environment("my_env", env_config={
            "full": True,
            "real_trace": False,
            "dp": "Summer 2019",
            "site": "caltech",
            "discrete": False,
            "seed": 123
        })
        .framework("tf2")
    )
    algo = train_config.build(env="my_env")

    for i in range(2):
        train_results = algo.train()

        algo.

def experiment(config):

    algo = train_config.build(env="my_env")
    print("algo built")
    for i in range(1):
        print("begin training algo")
        train_results = algo.train()
        print("done training algo")
        print(train_results['agent_timesteps_total'])
        print(train_results['custom_metrics'])
        print(train_results['episode_reward_max'])
        print(train_results['episode_reward_mean'])
        print(train_results['episode_reward_min'])

        # print(pretty_print(train_results))
        algo.save(checkpoint_dir)
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        tune.report({**train_results, "a": 2})
    algo.stop()

    # Manual Eval
    eval_config = (
        ppo.PPOConfig()
        .environment("my_env", env_config={
            "full": False,
            "real_trace": True,
            "dp": "Summer 2019",
            "site": "caltech",
            "discrete": False,
            "seed": 123
        })
    )
    eval_algo = eval_config.build(env="my_env")
    eval_algo.load_checkpoint(checkpoint_dir)
    env = eval_algo.workers.local_worker().env

    obs, info = env.reset()
    done = False
    eval_results = {"eval_reward": 0, "eval_eps_length": 0}
    while not done:
        action = eval_algo.compute_single_action(obs)
        next_obs, reward, done, truncated, info = env.step(action)
        eval_results["eval_reward"] += reward
        eval_results["eval_eps_length"] += 1
    eval_algo.stop()
    results = {**train_results, **eval_results}
    print(results)
    tune.report({**results, "a": 3})

import os

ray.init(num_cpus=3)
register_env("my_env", lambda config: get_env(**config)())

resources = ppo.PPO.default_resource_request(ppo.PPOConfig())

tuner = tune.Tuner(
    # experiment
    tune.with_resources(experiment, resources=resources),
    param_space={}
)
final_results = tuner.fit()

print(final_results)

2023-04-05 22:28:37,713	INFO worker.py:1553 -- Started a local Ray instance.
2023-04-05 22:28:38,690	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-04-05 22:28:38,691	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.


(experiment pid=31245) 2023-04-05 22:28:44,113	INFO algorithm_config.py:2888 -- Executing eagerly (framework='tf2'), with eager_tracing=tf2. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
(experiment pid=31245) 2023-04-05 22:28:44,280	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(experiment pid=31245) /home/ubuntu/ray_results/experiment_2023-04-05_22-28-38/experiment_3628c_00000_0_2023-04-05_22-28-38/
(experiment pid=31245) {}


(RolloutWorker pid=31309) 2023-04-05 22:28:49,385	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=31309) 2023-04-05 22:28:49,385	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.


(experiment pid=31245) algo built
(experiment pid=31245) begin training algo


(experiment pid=31245) 2023-04-05 22:29:52,511	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
experiment_3628c_00000,2023-04-05_22-30-37,False,,f1c1cd3291d043e886975cd22677bdeb,ip-172-31-19-228,1,172.31.19.228,31245,113.219,113.219,113.219,1680733837,0,,1,3628c_00000,0.00292063


2023-04-05 22:30:37,359	WARNING tensorboardx.py:224 -- You are trying to log an invalid value (ray/tune/_metric/config/output_compress_columns=['obs', 'new_obs']) via TBXLoggerCallback!


(experiment pid=31245) done training algo
(experiment pid=31245) 4000
(experiment pid=31245) {}
(experiment pid=31245) 3.93224650215404
(experiment pid=31245) 2.4892613019999303
(experiment pid=31245) 0.7552686244731401
(experiment pid=31245) Checkpoint saved in directory /home/ubuntu/ray_results/experiment_2023-04-05_22-28-38/experiment_3628c_00000_0_2023-04-05_22-28-38/


(RolloutWorker pid=37156) 2023-04-05 22:30:42,949	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=37156) 2023-04-05 22:30:42,949	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2023-04-05 22:30:45,247	ERROR tune.py:794 -- Trials did not complete: [experiment_3628c_00000]
2023-04-05 22:30:45,249	INFO tune.py:798 -- Total run time: 126.55 seconds (126.48 seconds for the tuning loop).


In [5]:
{result.log_dir: result.metrics_dataframe['a'] for result in final_results}


KeyError: 'a'

In [11]:
from ray.tune.logger import pretty_print

final_results.get_best_result().metrics['hist_stats']['episode_reward']

[3.0652830457648164,
 0.644091722092978,
 1.0565137843837575,
 3.6352404391060613,
 2.53774750717975,
 3.2504868899700825,
 1.9874198440814987,
 0.7707253657578359,
 2.797307384827306,
 2.5483773451139538,
 3.9048278693570166,
 0.5053711386258443]

In [17]:
final_results.get_best_result()

Result(metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.20000000298023224, 'cur_lr': 4.999999873689376e-05, 'total_loss': 0.118393734, 'policy_loss': -0.073018864, 'vf_loss': 0.18794881, 'vf_explained_var': 0.49560168, 'kl': 0.017318841, 'entropy': 76.6001, 'entropy_coeff': 0.0}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 480.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': 3.9048278693570166, 'episode_reward_min': 0.5053711386258443, 'episode_reward_mean': 2.2252826946884086, 'episode_len_mean': 288.0, 'episode_media': {}, 'episodes_this_iter': 12, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [3.065283045764